In [29]:
from openai import OpenAI
import json
import minsearch

### Load FAQ Documents

In [11]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [12]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

### Simple Retrieval

In [30]:
index = minsearch.Index(
    text_fields=['question', 'text', 'section'],
    keyword_fields=['course'],
)

In [31]:
index.fit(documents)

In [32]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query='the course has already started, can I still enroll?',
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [33]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Section The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th

### Simple Generation

In [13]:
client = OpenAI()

In [14]:
q = 'the course has already started, can I still enroll?'

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': q}]
)

In [15]:
response.choices[0].message.content

"Whether you can still enroll in a course that has already started depends on the specific policies of the institution or organization offering the course. Here are some steps you can take:\n\n1. **Check Enrollment Deadlines**: Look at the course information or the institution's website to see if there is an official deadline for enrollment. Some courses may allow late enrollments within a certain period.\n\n2. **Contact the Instructor**: Reach out directly to the instructor or course coordinator. Explain your situation and ask if it would be possible to join the course late. They may be able to make an exception or provide guidance on catching up.\n\n3. **Institution's Registrar Office**: Some universities and colleges have a registrar or admissions office that handles enrollments. Contact them to inquire about late enrollment options.\n\n4. **Online Courses**: For online courses, the rules may be more flexible. Platforms like Coursera, Udemy, and edX often allow you to join at any ti

### Augmented Generation

In [26]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT while answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}

CONTEXT: 
{context}
"""

In [34]:
# build context
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [35]:
print(context[:500])

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finis


In [38]:
prompt = prompt_template.format(question=q, context=context).strip()

In [40]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT. 
Use only the facts from the CONTEXT while answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?

CONTEXT: 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone projec

In [41]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{'role': 'user', 'content': prompt}]
)

In [42]:
response.choices[0].message.content

'Yes, even if the course has already started, you can still enroll and are eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects.'